<a href="https://colab.research.google.com/github/mawhy/OpenCV/blob/master/Image_Manipulation_and_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Processing CookBook
## Image Manipulation and Transformation

Image Manipulation and Transformation, is where you will learn how to use
different Python libraries (NumPy, SciPy, scikit-image, OpenCV, and Matplotlib) for image
manipulation and transformation. You will learn how to write Python code to do point
transforms (log/gamma transform, Gotham filter, colorspace transformation, and increasing
brightness/contrast) and geometric transforms (swirl transform, perspective transform, and
homography).



In [0]:
!git clone https://github.com/PacktPublishing/Python-Image-Processing-Cookbook.git
%cp -av "/content/Python-Image-Processing-Cookbook/Chapter 01/images/" "/content/"
%cp -av "/content/Python-Image-Processing-Cookbook/Chapter 01/models/" "/content/"
%rm -rf "/content/Python-Image-Processing-Cookbook"

### Colorspace Transformation: from RGB to Lab colorspace

In [0]:
%matplotlib inline
import numpy as np
from skimage import img_as_float
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb
from skimage.util import invert
from skimage.exposure import equalize_hist
import matplotlib.pylab as plt

im = imread('images/flowers.png')

im1 = rgb2lab(im)
im1[...,1] = im1[...,2] = 0
im1 = lab2rgb(im1)

plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(im), plt.axis('off'), plt.title('Original image', size=20)
plt.subplot(122), plt.imshow(im1), plt.axis('off'), plt.title('Gray scale image', size=20)
plt.show()

In [0]:
plt.figure(figsize=(20,5))
plt.subplot(131), plt.imshow(im), plt.axis('off'), plt.title('Original image', size=20)

im1 = rgb2lab(im)
im1[...,0] = im1[...,0] + 50 
im1 = lab2rgb(im1)

plt.subplot(132), plt.imshow(im1), plt.axis('off'), plt.title('Brighter image', size=20)

im1 = rgb2lab(im)
im1[...,0] = im1[...,0] - 50 
im1 = lab2rgb(im1)

plt.subplot(133), plt.imshow(im1), plt.axis('off'), plt.title('Darker image', size=20)

plt.show()

In [0]:
im1 = rgb2lab(im)
im1[...,0] = np.max(im1[...,0]) - im1[...,0]
im1 = lab2rgb(im1)

plt.figure(figsize=(20,6))
plt.subplot(131), plt.imshow(im), plt.axis('off'), plt.title('Original image', size=20)
plt.subplot(132), plt.imshow(im1), plt.axis('off'), plt.title('Inverted image (Lab)', size=20)
plt.subplot(133), plt.imshow(invert(im)), plt.axis('off'), plt.title('Inverted image (RGB)', size=20)
plt.show()

### Affine Transformation

In [0]:
import numpy as np
from scipy import ndimage as ndi
from skimage.io import imread
from skimage.color import rgb2gray

img = rgb2gray(imread('images/humming.png'))
w, h = img.shape
theta, lambda1 = np.pi/6, 0.5
mat_identity = np.array([[1,0,0],[0,1,0],[0,0,1]])
mat_reflect = np.array([[1,0,0],[0,-1,0],[0,0,1]]) @ np.array([[1,0,0],[0,1,-h],[0,0,1]])
mat_scale = np.array([[0.75,0,0],[0,1.25,0],[0,0,1]])
mat_rotate = np.array([[1,0,w/2],[0,1,h/2],[0,0,1]]) @ np.array([[np.cos(theta),np.sin(theta),0],[np.sin(theta),-np.cos(theta),0],[0,0,1]]) @ np.array([[1,0,-w/2],[0,1,-h/2],[0,0,1]])
mat_shear = np.array([[1,lambda1,0],[lambda1,1,0],[0,0,1]])
mat_all = mat_identity @ mat_reflect @ mat_scale @ mat_rotate @ mat_shear
plt.figure(figsize=(20,10))
img1 = ndi.affine_transform(img, mat_identity)
plt.subplot(231), plt.imshow(img1), plt.axis('off'), plt.title('Original image', size=20)
img1 = ndi.affine_transform(img1, mat_reflect) # offset=(0,h)
plt.subplot(232), plt.imshow(img1), plt.axis('off'), plt.title('First reflect', size=20)
img1 = ndi.affine_transform(img1, mat_scale)
plt.subplot(233), plt.imshow(img1), plt.axis('off'), plt.title('Then scale', size=20) 
img1 = ndi.affine_transform(img1, mat_rotate)
plt.subplot(234), plt.imshow(img1), plt.axis('off'), plt.title('Then rotate', size=20) 
img1 = ndi.affine_transform(img1, mat_shear)
plt.subplot(235), plt.imshow(img1), plt.axis('off'), plt.title('Then shear', size=20) 
plt.subplot(236), plt.imshow(ndi.affine_transform(img, mat_all)), plt.axis('off'), plt.title('All at once', size=20) 
plt.show()

### Non-linear Transformation

In [0]:
from skimage.io import imread
from skimage.transform import warp, PolynomialTransform
import matplotlib.pylab as plt

def swirl(xy, x0, y0, R):
    r = np.sqrt((xy[:,1]-x0)**2 + (xy[:,0]-y0)**2)
    a = np.pi*r / R
    xy[:, 1], xy[:, 0] = (xy[:, 1]-x0)*np.cos(a) + (xy[:, 0]-y0)*np.sin(a) + x0, -(xy[:, 1]-x0)*np.sin(a) + (xy[:, 0]-y0)*np.cos(a) + y0
    return xy

im = imread('images/lena.png')
print(im.shape)
im1 = warp(im, swirl, map_args={'x0':100, 'y0':100, 'R':250})
plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(im), plt.axis('off'), plt.title('Input image', size=20)
plt.subplot(122), plt.imshow(im1), plt.axis('off'), plt.title('Output image', size=20)
plt.show()

#print(im.shape, im1.shape)
t = PolynomialTransform()
x, y = np.mgrid[:im.shape[0], :im.shape[1]]
dst_indices = np.hstack((x.reshape(-1, 1), y.reshape(-1,1)))
dst_indices = dst_indices[np.random.choice(dst_indices.shape[0], 500),:]
src_indices = swirl(dst_indices, 100, 100, 250).astype(int)
t.estimate(dst_indices, src_indices, 8)
#print(src_indices)

img_warped = warp(im1, t, order=3, mode='constant',cval=float('nan'))
plt.figure(figsize=(10,10))
plt.imshow(img_warped), plt.axis('off')
plt.show()


### Elastic Transform

In [0]:
# https://gist.github.com/erniejunior/601cdf56d2b424757de5
# http://cognitivemedium.com/assets/rmnist/Simard.pdf
import numpy as np
import matplotlib.pylab as plt
from skimage.color import rgb2gray
from scipy.ndimage import gaussian_filter, map_coordinates 

def elastic_transform(image, alpha, sigma, random_state=None):
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dz = np.zeros_like(dx)
    #print(dx, dy, dz)
    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    print (x.shape)
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    distored_image = map_coordinates(image, indices, order=1, mode='reflect')
    return distored_image.reshape(image.shape)

img = plt.imread('images/lena.png')
img1 = elastic_transform(img, 8, 0.5)
plt.figure(figsize=(10,10))
plt.imshow(img1), plt.axis('off'), plt.title('Elastic Deformation', size=20)
plt.show()

### Perspective Projection and Homography

In [0]:
from skimage.transform import ProjectiveTransform
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np
import matplotlib.pylab as plt

im_src = (imread('images/humming.png'))
height, width, dim = im_src.shape
im_dst = np.zeros((height, width, dim))

pt = ProjectiveTransform()
src = np.array([[   0.,    0.],
       [height-1,    0.],
       [height-1,  width-1],
       [   0.,  width-1]])
dst = np.array([[ 295., 174.],
       [ 540., 146. ],
       [ 400., 777.],
       [  60., 422.]])

print(pt.estimate(src, dst))

x, y = np.mgrid[:height, :width]
dst_indices=np.hstack((x.reshape(-1, 1), y.reshape(-1,1))) 
src_indices = np.round(pt.inverse(dst_indices), 0).astype(int)
valid_idx = np.where((src_indices[:,0] < height) & (src_indices[:,1] < width) & (src_indices[:,0] >= 0) & (src_indices[:,1] >= 0))
dst_indicies_valid = dst_indices[valid_idx]
src_indicies_valid = src_indices[valid_idx]
im_dst[dst_indicies_valid[:,0],dst_indicies_valid[:,1]] = im_src[src_indicies_valid[:,0],src_indicies_valid[:,1]]
im_dst = im_dst.astype(np.uint8) 
plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(im_src, cmap='gray'), plt.axis('off'), plt.title('Source image', size=30)
plt.subplot(122), plt.imshow(im_dst, cmap='gray'), plt.axis('off'), plt.title('Destination image', size=30)
plt.show()

In [0]:
from skimage.transform import ProjectiveTransform, warp
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np
import matplotlib.pylab as plt

im_src = (imread('images/humming2.png'))
height, width, dim = im_src.shape
im_dst = np.zeros((height, width, dim))

pt = ProjectiveTransform()
src = np.array([[ 295., 174.],
       [ 540., 146. ],
       [ 400., 777.],
       [  60., 422.]])
dst = np.array([[   0.,    0.],
       [height-1,    0.],
       [height-1,  width-1],
       [   0.,  width-1]])

print(pt.estimate(src, dst))

x, y = np.mgrid[:height, :width]
dst_indices = np.hstack((x.reshape(-1, 1), y.reshape(-1,1))) 
src_indices = np.round(pt.inverse(dst_indices), 0).astype(int)
valid_idx = np.where((src_indices[:,0] < height) & (src_indices[:,1] < width) & (src_indices[:,0] >= 0) & (src_indices[:,1] >= 0))
dst_indicies_valid = dst_indices[valid_idx]
src_indicies_valid = src_indices[valid_idx]
im_dst[dst_indicies_valid[:,0],dst_indicies_valid[:,1]] = im_src[src_indicies_valid[:,0],src_indicies_valid[:,1]]
im_dst = im_dst.astype(np.uint8) 
plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(im_src, cmap='gray'), plt.axis('off'), plt.title('Source image', size=30)
plt.subplot(122), plt.imshow(im_dst, cmap='gray'), plt.axis('off'), plt.title('Destination image', size=30)
plt.show()
plt.figure(figsize=(10,10))
plt.imshow(warp(im_src.transpose(1,0,2), np.linalg.inv(pt.params), output_shape=(width,height)).transpose(1,0,2))
plt.show()

In [0]:
from skimage.transform import ProjectiveTransform
from skimage.io import imread
import numpy as np
import matplotlib.pylab as plt

im_src = (imread('images/man_moon.png'))
im_dst = imread('images/shutterstock.png')
height, width, dim = im_src.shape

pt = ProjectiveTransform()
src = np.array([[   0.,    0.],
       [height-1,    0.],
       [height-1,  width-1],
       [   0.,  width-1]])
dst = np.array([[ 41., 74.],
       [ 228., 72.],
       [ 192., 272.],
       [ 96., 272.]])

print(pt.estimate(src, dst))

x, y = np.mgrid[:im_dst.shape[0], :im_dst.shape[1]]
dst_indices=np.hstack((x.reshape(-1, 1), y.reshape(-1,1))) 
src_indices = np.round(pt.inverse(dst_indices), 0).astype(int)
valid_idx = np.where((src_indices[:,0] < height) & (src_indices[:,1] < width) & (src_indices[:,0] >= 0) & (src_indices[:,1] >= 0))
dst_indicies_valid = dst_indices[valid_idx]
src_indicies_valid = src_indices[valid_idx]
im_dst[dst_indicies_valid[:,0],dst_indicies_valid[:,1]] = im_src[src_indicies_valid[:,0],src_indicies_valid[:,1]]
plt.figure(figsize=(15,10))
plt.imshow(im_dst), plt.axis('off'), plt.title('Output Image', size=20)
plt.savefig('images/homography_out.png', bbox_in='tight', pad_in=0)
plt.close()
plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(im_src, cmap='gray'), plt.axis('off'), plt.title('Source image', size=30)
plt.subplot(122), plt.imshow(im_dst, cmap='gray'), plt.axis('off'), plt.title('Destination image', size=30)
plt.tight_layout()
plt.show()

In [0]:
from skimage.transform import ProjectiveTransform
from skimage.io import imread
import numpy as np
import matplotlib.pylab as plt

im_src = imread('images/painting.png')
im_dst = imread('images/graffiti.png')
im_mask = imread('images/graffiti_mask.png')
im_dst1 = np.copy(im_dst)
height, width, dim = im_src.shape
print(height, width, im_src.shape, im_dst.shape)

pt = ProjectiveTransform()
src = np.array([[   0.,    0.],
       [height-1,    0.],
       [height-1,  width-1],
       [   0.,  width-1]])
dst = np.array([[ 0., 0.],
       [im_dst.shape[0]-1, 0],
       [im_dst.shape[0]-1,  687],
       [ 0., 659]])
print(pt.estimate(src, dst))

im_dst_masked = im_dst & im_mask

x, y = np.mgrid[:im_dst.shape[0], :im_dst.shape[1]]
dst_indices = np.hstack((x.reshape(-1, 1), y.reshape(-1,1))) 
src_indices = np.round(pt.inverse(dst_indices), 0).astype(int)
valid_idx = np.where((src_indices[:,0] < height) & (src_indices[:,1] < width) & (src_indices[:,0] >= 0) & (src_indices[:,1] >= 0))
dst_indicies_valid = dst_indices[valid_idx]
src_indicies_valid = src_indices[valid_idx]
im_dst[dst_indicies_valid[:,0],dst_indicies_valid[:,1]] = im_src[src_indicies_valid[:,0],src_indicies_valid[:,1]]
im_dst &= (~im_mask) 
im_dst += im_dst_masked
plt.figure(figsize=(20,30))
plt.subplot(311), plt.imshow(im_src), plt.axis('off'), plt.title('Source image', size=30)
plt.subplot(312), plt.imshow(im_dst1), plt.axis('off'), plt.title('Destination image', size=30)
plt.subplot(313), plt.imshow(im_dst), plt.axis('off'), plt.title('Final image', size=30)
plt.show()

### Pencil Sketches from images

In [0]:
!pip install medpy

In [0]:
%matplotlib inline
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray
from skimage import util
from skimage import img_as_float
import matplotlib.pylab as plt
from medpy.filter.smoothing import anisotropic_diffusion
from skimage.filters import gaussian, threshold_otsu

def normalize(img):
    return (img-np.min(img))/(np.max(img)-np.min(img))

def sketch(img, edges):
    output = np.multiply(img, edges)
    output[output>1]=1
    output[edges==1]=1
    #output = normalize(output)
    return output

def edges_with_anisotropic_diffusion(img, niter=100, kappa=10, gamma=0.1):
    #img = gaussian(img, sigma=0.05)
    output = img - anisotropic_diffusion(img, niter=niter, kappa=kappa, gamma=gamma, voxelspacing=None, option=1)
    output[output > 0] = 1
    output[output < 0] = 0
    #output = np.clip(output, 0, 1)
    #thresh = threshold_otsu(output)
    #output = np.invert(output > thresh)
    return output

def edges_with_dodge2(img):
    img_blurred = gaussian(util.invert(img), sigma=5)
    output = np.divide(img, util.invert(img_blurred) + 0.001) # avoid division by zero
    print(np.max(output), np.min(output))
    output = normalize(output)
    thresh = threshold_otsu(output)
    output = output > thresh
    return output


def sketch_with_dodge(img):
    orig = img
    blur = gaussian(util.invert(img), sigma=20)
    result=blur/util.invert(orig) 
    result[result>1]=1
    result[orig==1]=1
    return result

# with DOG
def edges_with_DOG(img, k = 200, gamma = 1):
    sigma = 0.5
    output = gaussian(img, sigma=sigma) - gamma*gaussian(img, sigma=k*sigma)
    output[output > 0] = 1
    output[output < 0] = 0 
    return output

def sketch_with_XDOG(image, epsilon=0.01):
  """
  Computes the eXtended Difference of Gaussians (XDoG) for a given image. This 
  is done by taking the regular Difference of Gaussians, thresholding it
  at some value, and applying the hypertangent function the the unthresholded
  values.
  image: an n x m single channel matrix.
  epsilon: the offset value when computing the hypertangent.
  returns: an n x m single channel matrix representing the XDoG.
  """
  phi = 10

  difference = edges_with_DOG(image, 200, 0.98).astype(np.uint8)
  #difference = sketch(image, difference)
  #difference = normalize(difference)  

  for i in range(0, len(difference)):
    for j in range(0, len(difference[0])):
      if difference[i][j] >= epsilon:
        difference[i][j] = 1
      else:
        ht = np.tanh(phi*(difference[i][j] - epsilon))
        difference[i][j] = 1 + ht
  difference = normalize(difference)  
  return difference

def plot_sketches(img_file):
    img = rgb2gray(imread(img_file))
    plt.figure(figsize=(20,25))
    output_aniso = sketch(img, edges_with_anisotropic_diffusion(img))
    output_dog = sketch(img, edges_with_DOG(img, k=25))
    output_xdog = sketch_with_XDOG(img)
    output_dodge = sketch_with_dodge(img)
    output_dodge2 = sketch(img, edges_with_dodge2(img))
    plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95, hspace=0.05, wspace=0.05) 
    plt.subplot(321), plt.imshow(img), plt.axis('off')
    plt.title('Original Gray-scale image', size=20)
    plt.subplot(322), plt.imshow(output_aniso), plt.axis('off')
    plt.title('Sketch with anisotropic diffusion', size=20)
    plt.subplot(323), plt.imshow(output_dog), plt.axis('off')
    plt.title('Sketch with DOG', size=20)
    plt.subplot(324), plt.imshow(output_xdog), plt.axis('off')
    plt.title('Sketch with XDOG', size=20)
    plt.subplot(325), plt.imshow(output_dodge), plt.axis('off')
    plt.title('Sketch with Dodge', size=20)
    plt.subplot(326), plt.imshow(output_dodge2), plt.axis('off')
    plt.title('Sketch with Dodge2', size=20)
    #plt.show()
    plt.savefig(img_file.split('.')[0] + '_sketches_all.png', bbox_in='tight')

plt.gray()
img = 'images/bird.png'
plot_sketches(img)

### Cartoonizing

In [0]:
import cv2
import numpy as np
import matplotlib.pylab as plt

num_down = 2       # number of downsampling steps
num_bilateral = 7  # number of bilateral filtering steps

    
img = plt.imread("images/bean.png")
w, h, _ = img.shape
  
# downsample image using Gaussian pyramid
img_color = np.copy(img)
for _ in range(num_down):
    img_color = cv2.pyrDown(img_color)
 
# repeatedly apply small bilateral filter instead of
# applying one large filter
for _ in range(num_bilateral):
    img_color = cv2.bilateralFilter(img_color, d=9,
                                    sigmaColor=0.1,
                                    sigmaSpace=0.01)

# upsample image to original size
for _ in range(num_down):
    img_color = cv2.pyrUp(img_color)
    
img_color = cv2.resize(img_color, (img.shape[1], img.shape[0]))

# convert to grayscale and apply median blur
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
img_blur = cv2.medianBlur((255*img_gray).astype(np.uint8), 7) # convert float32 to uint8

# detect and enhance edges
img_edge = cv2.adaptiveThreshold((255*img_blur).astype(np.uint8), 255,
                                 cv2.ADAPTIVE_THRESH_MEAN_C, #ADAPTIVE_THRESH_GAUSSIAN_C, 
                                 cv2.THRESH_BINARY_INV,
                                 blockSize=11,
                                 C=0)

# convert back to color, bit-AND with color image
img_edge = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2RGB)

img_cartoon = cv2.bitwise_and((255*img_color).astype(np.uint8), img_edge)

# display
fig = plt.figure(figsize=(20,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 
plt.subplot(121)
plt.imshow(img)
plt.axis('off')
plt.title('Original Image', size=20)
plt.subplot(122)
plt.imshow(img_cartoon)
plt.axis('off')
plt.title('Cartoonized Image', size=20)

plt.show()

In [0]:
import numpy as np
import matplotlib.pylab as plt
from medpy.filter.smoothing import anisotropic_diffusion

img = plt.imread("images/bean.png")
output = img.copy()
for i in range(3):
    output[...,i] = anisotropic_diffusion(img[...,i], niter=50, kappa=0.1, gamma=0.05, voxelspacing=None, option=1)
plt.figure(figsize=(10,10))
plt.imshow(output), plt.axis('off')
plt.show()

### Pencil-Sketches, Color-Pencil-Sketches and Stylized-Color images with opencv-python

In [0]:
import cv2
import matplotlib.pylab as plt
src = cv2.imread('images/bird.png')
#dst = cv2.detailEnhance(src, sigma_s=10, sigma_r=0.15)
dst_sketch, dst_color_sketch = cv2.pencilSketch(src, sigma_s=50, sigma_r=0.05, shade_factor=0.05)
dst_water_color = cv2.stylization(src, sigma_s=50, sigma_r=0.05)
plt.figure(figsize=(20,14))
plt.subplot(221), plt.imshow(cv2.cvtColor(src, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Original', size=20)
plt.subplot(222), plt.imshow(dst_sketch, cmap='gray'), plt.axis('off'), plt.title('Pencil-Sketch', size=20)
plt.subplot(223), plt.imshow(cv2.cvtColor(dst_color_sketch, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Color-Pencil-Sketch', size=20)
plt.subplot(224), plt.imshow(cv2.cvtColor(dst_water_color, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Stylized-Color', size=20)
plt.show()

### Image stained glass with Voronoi tessellation

### Pointillism

In [0]:
!pip install pointillism

In [0]:
import pointillism as pt
point = pt.image(location='images/elephants.png', debug = True)
point.make('balanced')
point.display()
#point.save_out(location='images', suffix='basic test')

### Simulating Light art / long exposure

In [0]:
from glob import glob
import cv2
import numpy as np
import matplotlib.pylab as plt

def extract_frames(vid_file):
    vidcap = cv2.VideoCapture(vid_file)
    success,image = vidcap.read()
    i = 1
    success = True
    while success and i <= 200:
      cv2.imwrite('images/exposure/vid_{}.png'.format(i), image)
      success,image = vidcap.read()
      i += 1

extract_frames('images/godafost.mp4') #cloud.mp4

imfiles = glob('images/exposure/*.png')
nfiles = len(imfiles)
R1, G1, B1 = 0, 0, 0
for i in range(nfiles):
    image = cv2.imread(imfiles[i]).astype(float)
    (B, G, R) = cv2.split(image)
    R1 += R
    B1 += B
    G1 += G
R1, G1, B1 = R1 / nfiles, G1 / nfiles, B1 / nfiles
final = cv2.merge([B1, G1, R1])
plt.figure(figsize=(20,10))
plt.imshow(cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2RGB)), plt.axis('off')
plt.title('An input image', size=20)
plt.axis('off')
plt.show()
plt.figure(figsize=(20,10))
plt.imshow(cv2.cvtColor(final.astype(np.uint8), cv2.COLOR_BGR2RGB))
plt.axis('off'), plt.title('Output image', size=20)
plt.show()
cv2.imwrite('images/godafost.png', final)

### Extended Depth of Field with mahotas

In [0]:
!pip install mahotas

In [0]:
# https://mahotas.readthedocs.io/en/latest/edf.html
from glob import glob
import cv2
import numpy as np
import matplotlib.pylab as plt
import mahotas as mh
print(mh.__version__)

def create_image_stack(vid_file, n = 200):
    
    vidcap = cv2.VideoCapture(vid_file)
    success,image = vidcap.read()
    i = 0
    success = True
    h, w = image.shape[:2]
    imstack = np.zeros((n, h, w))
    while success and i < n:
      imstack[i,...] = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      success,image = vidcap.read()
      i += 1
    return imstack

image = create_image_stack('images/godafost.mp4') #cloud.mp4
stack,h,w = image.shape
plt.figure(figsize=(20,10))
plt.gray()
plt.imshow(image[0,...].astype(np.uint8)), plt.axis('off')
plt.title('An input image', size=20)
plt.axis('off')
plt.show()

focus = np.array([mh.sobel(t, just_filter=True) for t in image])
best = np.argmax(focus, 0)
image = image.reshape((stack,-1)) # image is now (stack, nr_pixels)
image = image.transpose() # image is now (nr_pixels, stack)
final = image[np.arange(len(image)), best.ravel()] # Select the right pixel at each location
final = final.reshape((h,w)) # reshape to get final result
plt.figure(figsize=(20,10))
plt.imshow(final.astype(np.uint8))
plt.axis('off'), plt.title('Output image', size=20)
plt.show()
cv2.imwrite('images/edf.png', final)

### Color Detection with opencv-python

In [0]:
#%matplotlib inline
import cv2
import numpy as np
import matplotlib.pylab as plt

bck = cv2.imread("images/fish_bg.png")
img = cv2.imread("images/fish.png")
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

mask = cv2.inRange(hsv, (5, 75, 25), (25, 255, 255))

## slice the orange fish
imask = mask>0
orange = np.zeros_like(img, np.uint8)
orange[imask] = img[imask]

yellow = img.copy()
hsv[...,0] = hsv[...,0] + 20
yellow[imask] = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)[imask]
yellow = np.clip(yellow, 0, 255)

bckfish = cv2.bitwise_and(bck,bck, mask=imask.astype(np.uint8))
nofish = img.copy()
nofish = cv2.bitwise_and(nofish,nofish, mask=(np.bitwise_not(imask)).astype(np.uint8))
nofish = nofish + bckfish

plt.figure(figsize=(20,12))
plt.subplots_adjust(0,0,1,0.9,0.01,0.075)
plt.subplot(221), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('original', size=20)
plt.subplot(222), plt.imshow(cv2.cvtColor(orange, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('only fish', size=20)
plt.subplot(223), plt.imshow(cv2.cvtColor(yellow, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('fish color changed', size=20)
plt.subplot(224), plt.imshow(cv2.cvtColor(nofish, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('transparent fish', size=20)
plt.suptitle('Color Detection with opencv-python', size=25)
plt.show()